In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import os
from torchvision import transforms
from sklearn.model_selection import train_test_split

class GenderDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        # แปลง gender เป็น numeric (0=male, 1=female)
        self.df['gender_num'] = self.df['gender'].map({'m': 0, 'f': 1})

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        try:
            img_name = os.path.join(self.img_dir, self.df.iloc[idx]['original_image'])
            image = Image.open(img_name).convert('RGB')
            
            # ตัดเฉพาะส่วนใบหน้า
            x, y = self.df.iloc[idx]['x'], self.df.iloc[idx]['y']
            dx, dy = self.df.iloc[idx]['dx'], self.df.iloc[idx]['dy']
            face = image.crop((x, y, x+dx, y+dy))
            
            if self.transform:
                face = self.transform(face)
            
            gender_label = torch.tensor(self.df.iloc[idx]['gender_num'], dtype=torch.long)
            return face, gender_label
            
        except Exception as e:
            print(f"Error loading image {img_name}: {str(e)}")
            # สร้าง dummy image กรณีโหลดไม่สำเร็จ
            dummy_image = torch.zeros((3, 224, 224))
            return dummy_image, torch.tensor(0)

class GenderClassifier(nn.Module):
    def __init__(self):
        super(GenderClassifier, self).__init__()
        
        # Feature extraction
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.3)
        )
        
        # Classification layers
        self.classifier = nn.Sequential(
            nn.Linear(256 * 28 * 28, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 2)  # 2 classes (male/female)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 256 * 28 * 28)
        x = self.classifier(x)
        return x

def train_gender_model(model, train_loader, val_loader, num_epochs=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', 
                                                    factor=0.1, patience=3,
                                                    verbose=True)
    
    best_val_acc = 0.0
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for i, (images, labels) in enumerate(train_loader):
            try:
                images = images.to(device)
                labels = labels.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                running_loss += loss.item()
                
                if (i + 1) % 100 == 0:
                    print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}]')
                    print(f'Loss: {running_loss/100:.4f}')
                    print(f'Accuracy: {100 * correct/total:.2f}%')
                    running_loss = 0.0
                    
            except Exception as e:
                print(f"Error in training batch {i}: {str(e)}")
                continue
        
        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for images, labels in val_loader:
                try:
                    images = images.to(device)
                    labels = labels.to(device)
                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    val_total += labels.size(0)
                    val_correct += (predicted == labels).sum().item()
                except Exception as e:
                    print(f"Error in validation: {str(e)}")
                    continue
        
        val_acc = 100 * val_correct/val_total
        print(f'Validation Accuracy: {val_acc:.2f}%')
        
        # Update learning rate
        scheduler.step(val_acc)
        
        # Save checkpoint if accuracy improves
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            print(f"Saving best model with accuracy: {val_acc:.2f}%")
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': val_acc,
            }, 'gender_classifier_best.pth')

def verify_paths(data_dir, img_dir):
    """ตรวจสอบความถูกต้องของ paths"""
    if not os.path.exists(data_dir):
        raise FileNotFoundError(f"Data directory not found: {data_dir}")
    if not os.path.exists(img_dir):
        raise FileNotFoundError(f"Image directory not found: {img_dir}")
    
    # ตรวจสอบไฟล์ fold_*.txt
    for i in range(5):
        file_path = os.path.join(data_dir, f'fold_{i}_data.txt')
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Fold file not found: {file_path}")

def main():
    # กำหนด paths
    data_dir = "all_path"  # โฟลเดอร์ที่มีไฟล์ fold_*.txt
    img_dir = os.path.join(data_dir, "faces")  # โฟลเดอร์ที่มีรูปภาพ
    
    try:
        # ตรวจสอบ paths
        verify_paths(data_dir, img_dir)
        
        # โหลดข้อมูล
        df_list = []
        for i in range(5):
            df = pd.read_csv(os.path.join(data_dir, f'fold_{i}_data.txt'), delimiter='\t')
            df_list.append(df)
        
        df = pd.concat(df_list, ignore_index=True)
        print(f"Loaded {len(df)} total records")
        
        # แบ่งข้อมูล
        train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
        print(f"Training set: {len(train_df)}, Validation set: {len(val_df)}")
        
        # Data transformation
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225])
        ])
        
        # สร้าง datasets
        train_dataset = GenderDataset(train_df, img_dir, transform=transform)
        val_dataset = GenderDataset(val_df, img_dir, transform=transform)
        
        # สร้าง data loaders
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
        
        # สร้างและ train model
        model = GenderClassifier()
        train_gender_model(model, train_loader, val_loader)
        
        # บันทึก model สุดท้าย
        torch.save(model.state_dict(), 'gender_classifier_final.pth')
        print("Training completed successfully")
        
    except Exception as e:
        print(f"Error in main: {str(e)}")

if __name__ == '__main__':
    main()

Error in main: Data directory not found: AdienceBenchmarkGenderAndAge
